In [ ]:
# ✅ 安裝套件
!pip install flask pyngrok sentence-transformers faiss-cpu openai==0.28 line-bot-sdk --quiet

# ✅ 匯入套件
import os, threading, time
import openai, faiss, numpy as np, requests, re
from flask import Flask, request, abort, jsonify
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
from linebot.v3 import WebhookHandler
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.exceptions import InvalidSignatureError
from google.colab import files

# ✅ 設定 OpenAI API Key（Colab 左邊輸入 OPENAI_API_KEY）
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 上傳並解析多益單字.txt
uploaded = files.upload()
filename = next(iter(uploaded))
entries = []
with open(filename, encoding="utf-8") as f:
    lines = f.read().splitlines()
for i in range(0, len(lines), 2):
    eng, ch = lines[i].strip(), (lines[i+1].strip() if i+1 < len(lines) else "")
    if eng and ch:
        entries.append({"eng": eng, "ch": ch})
print(f"✅ 共讀取 {len(entries)} 筆單字")

# ✅ 建立 FAISS 向量資料庫
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = [e["eng"] for e in entries]
embeds = model.encode(texts)
index = faiss.IndexFlatL2(embeds.shape[1])
index.add(np.array(embeds).astype("float32"))
print(f"✅ 向量資料庫已建立，含 {len(texts)} 筆向量")

# ✅ GPT Fallback 回應
def query_gpt_fallback(q):
    try:
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system", "content":"你是一位英文老師。"},
                {"role":"user", "content":f"解釋這個英文單字或句子：{q}"}
            ]
        )
        return res['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("[ERROR] GPT 呼叫失敗：", e)
        return "⚠️ GPT 無法回應，請稍後再試。"

# ✅ 判斷函式
def is_english_word(text):
    # 純英文單字（只含 a-z 或 A-Z）
    return re.fullmatch(r"[a-zA-Z]+", text) is not None

def is_english_sentence(text):
    # 英文句子，允許標點與空白，且詞數 > 1
    return re.fullmatch(r"[a-zA-Z\s,.!?;:'\"()-]+", text) is not None and len(text.split()) > 1

# ✅ Flask 初始化
app = Flask(__name__)

@app.route("/query", methods=["POST"])
def query_route():
    text = request.json.get("text", "").strip()
    if not isinstance(text, str) or not text:
        return jsonify({"error": "empty input"}), 400

    # 英文句子（多字）→ 直接用 GPT
    if is_english_sentence(text):
        gpt_result = query_gpt_fallback(text)
        return jsonify({"source": "gpt", "result": gpt_result})

    # 非純英文單字（有數字、中文、符號等）→ 直接用 GPT
    if not is_english_word(text):
        gpt_result = query_gpt_fallback(text)
        return jsonify({"source": "gpt", "result": gpt_result})

    # 純英文單字，使用 FAISS 找相似向量（模糊比對）
    query_vec = model.encode([text])
    distances, indices = index.search(np.array(query_vec).astype("float32"), 1)
    top_distance = distances[0][0]
    top_idx = indices[0][0]

    print(f"[DEBUG] 查詢詞：{text}, top_distance: {top_distance:.4f}")

    # 閾值 1.0 容許拼錯，模糊比對
    if top_distance < 1.0:
        result = entries[top_idx]
        return jsonify({"source": "faiss", "result": result})
    else:
        gpt_result = query_gpt_fallback(text)
        return jsonify({"source": "gpt", "result": gpt_result})

# ✅ Line Bot 設定
LINE_CHANNEL_ACCESS_TOKEN = os.getenv("LINE_CHANNEL_ACCESS_TOKEN")
LINE_CHANNEL_SECRET = os.getenv("LINE_CHANNEL_SECRET")

config = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

if not LINE_CHANNEL_ACCESS_TOKEN or not LINE_CHANNEL_SECRET:
    raise ValueError("❌ 請先設定 LINE_CHANNEL_ACCESS_TOKEN 和 LINE_CHANNEL_SECRET 環境變數")

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers.get('X-Line-Signature')
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_text = event.message.text
    try:
        res = requests.post(f"{public_url}/query", json={"text": user_text})
        data = res.json()
        if data.get("source") == "faiss":
            reply = f"{data['result']['eng']}：{data['result']['ch']}"
        else:
            reply = data["result"]
    except Exception as e:
        print("回傳錯誤：", e)
        reply = "⚠️ 查詢失敗，請稍後再試。"

    req = ReplyMessageRequest(
        reply_token=event.reply_token,
        messages=[TextMessage(text=reply)]
    )
    with ApiClient(config) as client:
        MessagingApi(client).reply_message_with_http_info(req)

# ✅ Flask 執行程式
def run():
    app.run(port=5000)

# ✅ 關閉舊 ngrok + 啟動新 ngrok
os.system("pkill -f ngrok")
time.sleep(1)
public_url = ngrok.connect(5000).public_url
print(f"🚀 請將以下網址貼到 LINE Webhook：{public_url}/callback")

# ✅ 啟動 Flask
threading.Thread(target=run).start()
time.sleep(2)